In [3]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

import tensorflow.keras as keras
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [4]:
# https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/
# https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/custom/15844304.pdf

In [5]:
# !pip install tensorflow_text

In [6]:
import pandas as pd
import warnings


warnings.filterwarnings('ignore')

class Train_Data():
    
    '''
    Train_Data class Loads the data from multiple Training JSON files in Pandas Dataframes
    '''
   
    def __init__(self, path , filenames):
        '''
            Parameters
            ------------
            path: directory where English to SQL translation JSON are placed
            filenames: Json file names containing Train data
        '''
        self.path = path
        self.filenames = filenames
        self.df_train = pd.DataFrame()
        for f in self.filenames:
            print("Reading file at path", self.path+f)
            try:
                df = pd.read_json(self.path + f)
                if len(self.df_train) == 0:
                    self.df_train = df
                else:
                    self.df_train = self.df_train.append(df)
                print("{} Rows in Total".format(len(self.df_train)))
            except Exception as e:
                print("Got error while Reading file : " , e)
                
    @property          
    def questions(self):
        ''' 
        Returns
        ------------
        Returns English Questions in Dataframe Rows as List
        '''
        return self.df_train.question.values.tolist()
    
    @property
    def sql(self):
        '''
        Returns
        ------------
        Returns SQL in Dataframe Rows as List
        '''
        return self.df_train['query'].values.tolist()
    
    @property          
    def question_tokens(self):
        ''' 
        Returns
        ------------
        Returns English Question Tokens in Dataframe Rows as List
        '''
        
        return self.df_train['question_toks'].values.tolist()
    
    @property
    def sql_tokens(self):
        ''' 
        Returns
        ------------
        Returns SQL Query Tokens in Dataframe Rows as List
        '''
        return self.df_train['query_toks'].values.tolist()
    
    def get_special_characters(self,list_of_text):
        '''
        Parameters
        ------------
        list_of_text: Input List of Text 
        Returns
        ------------
        Provides list of Special Characters in the text
        '''
        return list(set(Preprocess.special_char(''.join([''.join(ele) for ele in list_of_text]))))
    
    def get_vocab_size(self, list_of_text):
        """
        Parameters
        ------------
        list_of_text: Input List of Text 
        
        Returns
        ------------
        Vocabulary size or unique words in the corpus
        """
        word_list = []
        for sentence in list_of_text:
            for word in sentence.split():
                word = word.lower().strip()
                if word not in word_list:
                    word_list.append(word)
        return len(word_list), word_list
        
    
    
    
    
    
    
#################################
#CONSTANTS

EOS = '[end]'
SOS = '[start]'  
    
    
    
    
    
#################################################################################################################
    
import re
import tensorflow as tf
import tensorflow_text as tf_text

class Preprocess():
    
    '''
    Preprocess class cleans and standardize the data, add SOS-EOS tags to the data
    '''
    
    def __init__(self,x , text):
        """
            Parameters
            ------------
            text : Input string
            Runs the text processing steps
            
        """
        self.x = x
        self.processed_text = self.run_pipeline(text)
    
    def text_standardize(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Unicode normalization using NFKD method
            -   Lower Case text
        
        """
        # text = tf_text.normalize_utf8(text, 'NFKD')
        text = tf.strings.lower(text)
        return text
    
    def text_whitespace(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Remove $ and \\ special characters 
            -   Add space around punctations
            -   Remove spaces around sentences
        
        """
        text = tf.strings.regex_replace(text, '[$\\\\]', '')
        text = tf.strings.regex_replace(text, '[.?!,¿()*:@]', r' \0 ')
        text = tf.strings.strip(text)
        return text
    
    def add_SOS_EOS(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Add <SOS> and <EOS> tags to each sentence
        
        """
        if self.x == 'i':
            return text
        if self.x == 'o':
            text = tf.strings.join([text, EOS], separator=' ')
            return text
        if self.x == 's_o':
            text = tf.strings.join([SOS,text], separator=' ')
            return text
        
        return text
    
    @classmethod
    def special_char(cls, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Special Characters found in Text using Regular Expression
        """
        return re.findall(r'[\W]',text.replace(' ',''))
    
    
    def run_pipeline(self,text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            Executes series of Text pre processing functions
        
        """
        text = self.text_standardize(text)
        text = self.text_whitespace(text)
        text = self.add_SOS_EOS(text)
        self.text = text
        return self.text
    
    
class Features():
    '''
    Extracts text features from data
    '''
    def __init__(self,x):
        self.x = x
        
    def tf_lower_and_split_punct(self,text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            Standardized Text
        
        """
        return Preprocess(self.x, text).processed_text
        
    def vectorizor(self, document, max_vocab_size):
        """
            Parameters
            ------------
            document : Collection of sentences
            max_vocab_size : No of words in document used for TextVectorization
            
            Returns
            ------------
            TextVectorization object
        
        """
        text_processor = tf.keras.layers.TextVectorization( standardize = self.tf_lower_and_split_punct, max_tokens = max_vocab_size)
        text_processor.adapt(document)
        print("Sample Vocabulary",text_processor.get_vocabulary()[:10])
        return text_processor

In [7]:
try:
    del(o)
except:
    pass


o = Train_Data('D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/',['train_spider.json','train_others.json'])

o.sql[0]

Reading file at path D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/train_spider.json
7000 Rows in Total
Reading file at path D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/train_others.json
8659 Rows in Total


'SELECT count(*) FROM head WHERE age  >  56'

In [174]:
input_text_processor = Features('i').vectorizor(o.questions , 7000)
output_text_processor = Features('o').vectorizor(o.sql , 7000)
output_shifted_text_processor = Features('s_o').vectorizor(o.sql , 7000)


Sample Vocabulary ['', '[UNK]', 'the', 'of', '?', '.', 'what', 'are', 'and', 'in']
Sample Vocabulary ['', '[UNK]', '.', 't1', 't2', '=', 'select', 'from', 'as', '[END]']
Sample Vocabulary ['', '[UNK]', '.', 't1', 't2', '=', 'select', 'from', 'as', '[START]']


In [70]:
max_input_len = input_text_processor(o.questions[:500]).shape[-1]
max_input_len

27

In [71]:
max_out_len = output_text_processor(o.sql[:500]).shape[-1]
max_out_len

80

In [8]:
import numpy as np

embeddings_dictionary = dict()

emb = 'D:/DS/Learnin/Essex MS/CE888/embeddings/glove.6B.100d.txt'
glove_file = open(emb, encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [73]:
num_words = input_text_processor.vocabulary_size()
EMBEDDING_SIZE = 100
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))
for index,word in enumerate(input_text_processor.get_vocabulary()):
    # if word:
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [74]:
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
import numpy as np

embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [75]:
len_of_input_sentences = len(o.questions[:500])
num_words_output = output_text_processor.vocabulary_size()+1
############################
decoder_targets_one_hot = np.zeros((
        len_of_input_sentences,
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)
decoder_targets_one_hot.shape

(500, 80, 3821)

In [76]:
LSTM_NODES =256
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [77]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [78]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [79]:
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [80]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [81]:
# !pip install pydot

In [83]:
d=output_shifted_text_processor(o.sql[:500])

In [84]:
for i, d in enumerate(d):
    # print(i)
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [85]:
r = model.fit(
    [input_text_processor(o.questions[:500]), output_shifted_text_processor(o.sql[:500])],
    decoder_targets_one_hot,
    batch_size=64,
    epochs=10,
    validation_split=0.1,
)

Epoch 1/10
8/8 [==============================] - 16s 2s/step - loss: 5.9967 - accuracy: 0.1168 - val_loss: 3.4204 - val_accuracy: 0.8480
Epoch 2/10
8/8 [==============================] - 17s 2s/step - loss: 2.7025 - accuracy: 0.7618 - val_loss: 1.5185 - val_accuracy: 0.8480
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: 1.7776 - accuracy: 0.7618 - val_loss: 1.3719 - val_accuracy: 0.8480
Epoch 4/10
8/8 [==============================] - 17s 2s/step - loss: 1.6164 - accuracy: 0.7635 - val_loss: 1.0992 - val_accuracy: 0.8480
Epoch 5/10
8/8 [==============================] - 17s 2s/step - loss: 1.4207 - accuracy: 0.7653 - val_loss: 0.9478 - val_accuracy: 0.8605
Epoch 6/10
8/8 [==============================] - 18s 2s/step - loss: 1.3050 - accuracy: 0.7743 - val_loss: 0.8658 - val_accuracy: 0.8605
Epoch 7/10
8/8 [==============================] - 17s 2s/step - loss: 1.2202 - accuracy: 0.7771 - val_loss: 0.8333 - val_accuracy: 0.8765
Epoch 8/10
8/8 [==================

In [86]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)


In [87]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [88]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [89]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [90]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [91]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [125]:
output_shifted_text_processor(['<sos>']).numpy()

array([[9, 1]], dtype=int64)

In [138]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1
    eos = 9
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == 10:
            break

        word = ''

        if idx > 0:
            word = output_text_processor.get_vocabulary()[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [139]:
output_shifted_text_processor(EOS)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([9, 1], dtype=int64)>

In [140]:
np.zeros((1, 1))

array([[0.]])

In [141]:

# i=input_text_processor(o.questions[:500])[2]
# translation = translate_sentence(i)
# print('-')
# print('Input:', o.questions[2])
# print('Response:', translation)

In [142]:
i = np.random.choice(len(o.questions[:500]))
input_seq = input_text_processor(o.questions[:500])[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', o.questions[i])
print('Response:', translation)

-
Input: Show names for all aircrafts with distances more than the average.
Response: [END] select select from from from from from from from


In [113]:
input_text_processor(o.sql[:500])[0:1]

<tf.Tensor: shape=(1, 79), dtype=int64, numpy=
array([[ 783,   57,  191,    1,  190,   58, 1471,   94,   86, 3713, 3721,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int64)>

In [109]:
input_text_processor(o.questions[:500])[2]

<tf.Tensor: shape=(27,), dtype=int64, numpy=
array([  30,    2, 1492,   64,   21,   12,    8,  267,    3,   29,   68,
          5,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int64)>

In [2]:
import os, sys

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [15]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [16]:
input_sentences = [x.decode('utf-8') for x in list(Preprocess('i',o.questions[:500]).processed_text.numpy())]
output_sentences = [x.decode('utf-8') for x in list(Preprocess('o',o.sql[:500]).processed_text.numpy())]
output_sentences_inputs = [x.decode('utf-8') for x in list(Preprocess('s_o',o.sql[:500]).processed_text.numpy())]

In [17]:
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

what are the names of all stations that have more than 10 bikes available and are not located in san jose ?
select t1 . name from station as t1 join status as t2 on t1 . id  =  t2 . station_id group by t2 . station_id having avg ( bikes_available )   >  10 except select name from station where city  =  "san jose" [end]
[start] select t1 . name from station as t1 join status as t2 on t1 . id  =  t2 . station_id group by t2 . station_id having avg ( bikes_available )   >  10 except select name from station where city  =  "san jose"


In [18]:
input_sentences[172]

'what are the names of all stations that have more than 10 bikes available and are not located in san jose ?'

In [19]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 500
num samples output: 500
num samples output input: 500


In [20]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 559
Length of longest sentence in input: 26


In [21]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 334
Length of longest sentence in the output: 80


In [22]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

encoder_input_sequences.shape: (500, 26)
encoder_input_sequences[172]: [  0   0   0   0   0   3   4   1  12   2   7  34   9  11  27  18  97  94
 107   5   4  41 120  10 138 233]


In [23]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (500, 80)
decoder_input_sequences[172]: [ 12   2   4   1  19   3  32   9   4  16  40   9   5  18   4   1  31   6
   5   1  50  20  10   5   1  50  37  29   7  79   8  25  98  95   2  19
   3  32  13  26   6 121 184   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]


In [24]:
word2idx_outputs['[start]']
word2idx_outputs['[end]']

11

In [25]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [26]:
print(embedding_matrix[539])

[ 0.73132998 -0.66733998  0.74194002 -0.18158001  0.29214999  0.43759999
  0.23856001 -0.24066    -0.23841    -0.38927001  1.59239995 -0.063928
  0.54516     0.43784001 -0.70384002  0.28386     1.41639996 -0.093074
  0.17941999  0.38483    -0.025013    0.03748     0.20721     0.12104
 -0.69294    -0.13064     0.30320999 -0.49383     0.15166    -0.19228999
  0.71739    -0.18375     0.15871     1.19599998 -0.37094    -0.069602
 -0.57428998 -0.0034853  -0.2502     -0.63968998 -0.0029819  -0.11249
 -0.29874     0.073009   -0.16331001 -0.91198999  0.14664    -0.82380998
  0.71890998 -0.53586     0.81794    -0.085717    0.06512     0.75410998
  0.72373998 -2.01740003 -0.18441001  0.12034     1.44529998  0.95161998
 -0.21149001  0.63142002 -0.11256     0.10271     0.5273     -0.58968002
  0.37007999  0.39436001  1.00010002 -0.40781999 -0.10999     0.52677
 -0.19093999  0.059869    0.18929     0.19038001 -0.089262   -0.50409001
 -0.89447999 -0.40553001  0.65720999  0.26298001  0.42081001  0.22

In [27]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)


In [28]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)

In [29]:
decoder_targets_one_hot.shape


(500, 80, 335)

In [30]:
for i, d in enumerate(output_input_integer_seq):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [31]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [32]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [33]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [34]:
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [37]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=64,
    epochs=100,
    validation_split=0.1,
)

Epoch 1/100
8/8 [==============================] - 2s 299ms/step - loss: 0.2797 - accuracy: 0.1996 - val_loss: 0.3117 - val_accuracy: 0.1060
Epoch 2/100
8/8 [==============================] - 2s 303ms/step - loss: 0.2741 - accuracy: 0.2006 - val_loss: 0.3081 - val_accuracy: 0.1065
Epoch 3/100
8/8 [==============================] - 2s 281ms/step - loss: 0.2658 - accuracy: 0.2015 - val_loss: 0.3347 - val_accuracy: 0.1055
Epoch 4/100
8/8 [==============================] - 2s 280ms/step - loss: 0.2628 - accuracy: 0.2023 - val_loss: 0.2985 - val_accuracy: 0.1067
Epoch 5/100
8/8 [==============================] - 2s 275ms/step - loss: 0.2496 - accuracy: 0.2047 - val_loss: 0.3020 - val_accuracy: 0.1072
Epoch 6/100
8/8 [==============================] - 2s 279ms/step - loss: 0.2452 - accuracy: 0.2060 - val_loss: 0.3132 - val_accuracy: 0.1060
Epoch 7/100
8/8 [==============================] - 2s 284ms/step - loss: 0.2417 - accuracy: 0.2071 - val_loss: 0.3120 - val_accuracy: 0.1067
Epoch 8/100
8

In [38]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [39]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [40]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [41]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [42]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [43]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['[start]']
    eos = word2idx_outputs['[end]']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [44]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: for each city ,  list their names in decreasing order by their highest station latitude .
Response: [start] select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select select
